# 第四次课后练习

**负责助教：王瑞环**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第四次课后练习+姓名+学号.ipynb, 例如 第四次课后练习+张三+1000000000.ipynb</span>

<span style="color:red; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第四次课后练习+张三+1000000000+#.ipynb</span>

<span style="color:red; font-weight:bold;">本次课同时发布课后练习和选做题，提交时请注意区分提交通道</span>

# 第零部分 代码理解

请认真阅读代码，理解代码的功能，先写出预想的结果。运行并检验结果是否如预期。如果不如预期，请分析理解其中的原因

## **0.1** 类变量与实例变量

In [3]:
class Config:
    data = {}
    
    def __init__(self, name):
        self.name = name
        self.data['key'] = 'default'

a = Config('a')
b = Config('b')
print(a.data is b.data)  
print(a.__class__.data is Config.data) 
a.data = {'key': 'custom'}
#a.data['key'] = 'custom'
print(b.data['key'])

True
True
default


True

True

default

按照：实例 -> 类 -> 父类的顺序查找attr

前两个是因为类属性共享。a.__class__指向的就是Config类。第三个是在实例a中创建了新的实例属性字典data（因为这是赋值操作而非内容的改变），不影响类属性的data。

如果改成a.data['key'] = 'custom'，那么b.data就会修改。（因为这是修改字典的内容而不是赋值，就不会创建实例变量）

## **0.2** 单例模式

In [31]:
class Singleton1:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls.value = 100
        return cls._instance

a = Singleton1()
a.value = 200
b = Singleton1()
print(a.value) 
print(b.value) 
print(Singleton1.value)   

200
200
100


200

200

100

__new__中可看出是单例模式。类变量 _instance <mark>存储唯一实例</mark>，所有<mark>实例</mark>都共享它。

创建a实例后，cls.value = 100(类属性)，Singleton1._instance不是None了；然后赋值a.value,相当于创建新的a实例属性value，不影响类属性Singleton1.value（仍是100）；再创建b实例，因为Singleton1._instance不是None，所以b直接指向这个_instance，即b指向a所指的同一个实例，b和a共享同一个实例（故也拥有实例属性value=200），因此b.value=a.value=200

In [ ]:
class Singleton2:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        cls.value = 100
        return cls._instance

a = Singleton2()
a.value = 200
b = Singleton2()
print(a.value)
print(b.value)
print(Singleton2.value) 

200
200
100


200

200

100

几乎同上，只是每次创建实例时都会将类属性Singleton2.value赋值成100（并非实例属性），但是b与a是同一个实例（单例模式）。所以b.value(实例属性)=a.value(实例属性)=200

In [33]:
class Singleton3:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        cls._instance.value = 100  
        return cls._instance

a = Singleton3()
a.value = 200
b = Singleton3()
print(a.value)
print(b.value)
print(Singleton3.value) 

100
100


AttributeError: type object 'Singleton3' has no attribute 'value'

100

100

Error

多加的这一行
```python 
cls._instance.value = 100  
```
导致创建实例时都会将实例属性修改为100。因此顺序为：先创建实例a，实例属性a.value = 100,然后是修改实例属性为a.value=200,然后创建实例b，因为cls._instance != None (实际等于实例a)，所以只执行修改实例属性a.value=100并将实例a返回给b，因此b.value = a.value = 100.由于从来没有定义类属性Singleton3.value所以报错。

## **0.3** 函数闭包与工厂函数

In [4]:
functions = []
for i in range(3):
    def func():
        return i
    functions.append(func)

# 如果这里写： i = 6， 那么输出结果是：[6, 6, 6]

print([f() for f in functions])

[2, 2, 2]


[2, 2, 2]

函数闭包：内部函数引用外部变量，而不是拷贝外部变量，所以return i 是返回外部这个i的引用。

#### <mark> Python 的后期绑定指函数或闭包在其被调用时才确定它引用的变量值，而不是在定义时就确定。这种行为特别容易在使用循环和闭包时出现，因为闭包会捕获变量的引用，而不是变量的当前值。 </mark>
#### <mark> Python 使用后期绑定来处理函数内部对外部变量（如循环变量、函数参数等）的引用。当定义函数时，它并没有立刻决定这些外部变量的具体值，而是等到函数调用时才决定。</mark>
每次循环中都会定义函数 func()。每次定义 func() 时，它会闭包（closure）捕获变量 i，但是由于 Python 的延迟绑定行为，这个闭包并不会在定义时捕获 i 的值，而是等到函数被调用时再去查找i的值。每次循环时，都会将func() 函数添加到 functions中。然而，因为 func() 是引用了变量 i，它并没有捕获 i 在定义时的值，而是始终引用 i，直到调用时获取i的值。故最后在调用函数时，所有的函数都会引用 i。但到此时 i = 2 了。

修改方式：第三行改为：
```python
def func(i=i):
```
这样传入默认参数使得函数在定义时捕获i的当前值。

In [ ]:
functions = []
for i in range(3):
    def factory(i): 
        def func():
            return i 
        return func
    functions.append(factory(i)) 

print([f() for f in functions]) 

[0, 1, 2]

工厂函数，传参时传入了当时外部i的值。

这里就在循环中捕获了i的值，在循环中调用factory(i)。故函数闭包可以捕获当前i的值，每个func捕获了不同的i，故输出为[0,1,2]

## **0.4** 多进程编程

由于 Jupyter Notebook 无法直接运行多进程程序，我们采用一种迂回的方法：使用`%%writefile`将要运行的程序写入到一个`.py`文件中，再获取运行该文件所得到的结果

需要判断
- 输出是否稳定（即多次运行是否能保证输出结果一致）
  - 如果稳定，预测输出结果
  - 如果不稳定，预测所有输出结果的可能情况
- 代码大致的运行时间

In [ ]:
%%writefile multiprocessing_script.py
from multiprocessing import Pool
import time
import random

def square(x):
    time.sleep(1)
    return x*x

if __name__ == "__main__":
    
    with Pool(5) as p:
        start_time = time.time()
        result = p.map(square, range(20))
        print("Result:", result)
        end_time = time.time()
    print(f"All done, time taken: {end_time - start_time:.2f}")

In [ ]:
import subprocess
result = subprocess.run(
    ["python", "multiprocessing_script.py"], 
    capture_output=True, text=True)
print(result.stdout)

Result: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]

All done, time taken: 4s (约)

进程池中只有5个进程，故每次同时进行5个任务。共20个任务，每个都约为1s，故约20/5=4s完成。结果是稳定的，因为map会等待所有任务完成并按顺序收集结果。（解释：<mark>map() 的实现方式使用了一个阻塞机制，确保只有当所有的任务都完成时，它才会将结果返回给用户。而且，无论任务在执行过程中是先完成还是后完成，map() 会根据任务的原始输入顺序来组织结果。</mark>）

## **0.5** 生成器函数与 send() 操作

生成器函数（Generator Function）是一种通过 yield 语句逐步生成值的特殊函数。与普通函数不同，生成器函数通过 yield 暂停执行，保存当前状态，允许后续恢复执行。send() 是生成器的一个方法，用于向生成器内部传递数据，实现对生成器函数发送信息与控制。

示例：动态调整步长的计数器

In [3]:
def dynamic_counter(start=0):
    current = start
    step = 1
    while True:
        # 通过yield返回当前值，并接收外部send的step值
        new_step = yield current
        if new_step is not None:  # 如果接收到新step，更新
            step = new_step
        current += step

In [ ]:
# 初始化生成器
counter = dynamic_counter(5)

# 启动生成器，获取初始值
print(next(counter))  

# 发送步长3，并获取下一个值
print(counter.send(3))  

# 继续迭代，使用当前步长3
print(next(counter))    

# 发送步长1，并获取下一个值
print(counter.send(1)) 

5
8
11
12


5

8

11

12

```python
# 启动生成器，获取初始值
print(next(counter)) 
```
上面这行可以改为下面这行：
```python
print(counter.send(None)) 
```
------

```python
new_step = yield current
```
这行作用是如果运行到此处的yield，先返回此时的current值，然后立刻等待send给new_step赋值并跳到下一次yield，进而会再返回新的current并等待更下一次的send（如果没有send直接用next跳过，那么相当于send了None）。因此刚开始要先启动生成器，即先运行到第一次yield，已经先返回此时的current = 5，然后才能等待send（直接send会报错，因为send只能在yield处使用）

第一次send了3，此时step改成3，current=5+3=8，到while的第二轮，先返回此时current=8，然后等待send

然后直接next，相当于send了None并跳到下一次yield，故step还是3，current=8+3=11，到while的第三轮，先返回此时current=11，然后等待send

最后send了1，此时step改成1，current=11+1=12，到while的第四轮，先返回此时current=12，然后等待send

## **0.6** `yield`递归生成器

In [3]:
from collections.abc import Iterable
nested = [[[1], 2], 3, 4, [5, [[6, 7]]]]

In [4]:
def flatten1(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)): 
            for subitem in flatten1(item):
                yield subitem
        else:
            yield item

for i in flatten1(nested):
    print(i)

1
2
3
4
5
6
7


1 

2 

3 

4 

5 

6 

7

isinstance作用：某个变量是否为后面这些类之一的实例 ； 这里bytes是不可变的二进制数据类型，表示原始的二进制数据。

这里是递归的生成这个可迭代对象的所有数（因为这个list全是整型）：如果为不可迭代对象则直接生成，如果是可迭代对象（并且不能是字符串和bytes），那么递归下去，迭代这个对象，对子对象接着做，直到最后一层全是不可迭代对象。故输出为按列表内部按前序顺序生成。

In [5]:
def flatten2(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            yield from flatten2(item)
        else:
            yield item

for i in flatten2(nested):
    print(i)

1
2
3
4
5
6
7


1

2

3

4

5

6

7

yield from 作用：委托给子迭代器。因此仿上这是递归的生成这个list的所有数（因为这个list全是整型）：如果为不可迭代对象则直接生成，如果是可迭代对象（并且不能是字符串和bytes），那么递归下去，迭代这个对象，对子对象接着调用生成器作为大生成器的结果，直到最后一层全是不可迭代对象。故输出为按列表内部按前序顺序生成。

In [6]:
def flatten3(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            flatten3(item)
        else:
            yield item

for i in flatten3(nested):
    print(i)

3
4


3

4

这里$flatten3(item)$并没有赋值给某个生成器，也没有yield from，故不造成任何生成(yield!=return)。因此所有生成都产生自不可迭代对象，即第一层不是list的那些：3，4.

In [7]:
def flatten4(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            yield flatten4(item)
        else:
            yield item

for i in flatten4(nested):
    print(i)

<generator object flatten4 at 0x10533f970>
3
4
<generator object flatten4 at 0x10533f970>


某个生成器地址

3

4

某个生成器地址(与上面相同的地址)

这里$flatten4(item)$是直接yield，故在生成器$flatten4(nested)$中会有某些生成器本身作为对象，在最后的循环迭代中只会遍历到这些对象而不会递归地迭代。因此对第一层不是list的那些：3，4正常生成；而第一层是list的那些：会生成迭代器地址。两个迭代器地址相同是因为python解释器优化：python可能会优化相同模式的 flatten4(item) 生成器对象，导致它们在内存中被复用。


# 第一部分 代码填空

## **1.1** 工厂模式

补充`SimpleFactory.create_product`函数，其接受一个字符串`product_type`作为参数，表示用户要求制作的产品
- 当`product_type`为`"A"`或`"Product A"`时，生产出一个`ProductA`对象并返回
- 当`product_type`为`"B"`或`"Product B"`时，生产出一个`ProductB`对象并返回
- 否则，抛出异常，表明商品无法制作

In [ ]:
class ProductA:
    pass

class ProductB:
    pass

class SimpleFactory:
    @staticmethod
    def create_product(product_type: str):
        if product_type in ['A','Product A']: return ProductA() # 短路特性: 不能写：x == 'A' or 'Product A'
        if product_type in ['B','Product B']: return ProductB()
        raise ValueError(f"Unknown product type: {product_type}")

factory = SimpleFactory()

product_a1 = factory.create_product("A")
product_a2 = factory.create_product("Product A")
assert isinstance(product_a1, ProductA)
assert isinstance(product_a2, ProductA)
product_b1 = factory.create_product("B")
product_b2 = factory.create_product("Product B")
assert isinstance(product_b1, ProductB)
assert isinstance(product_b2, ProductB)


try: 
    product_c = factory.create_product("C")
except: 
    pass
else:
    assert False

## **1.2** 观察者模式/发布-订阅模式

简单的来说，观察者模式就是由多位观察者(Observer)观察一个对象(Subject)，被观察对象发生了某种变化时，所有观察他的对象得到通知并被自动更新

如学生-教学网可以视为一组观察者-被观察对象：某课程在教学网上**发布**（公告、作业等）时，选课同学（**订阅/观察**该课程的学生）可以收到**通知**（作业截止日期、公告内容等），所以观察者模式又称为发布-订阅模式

GUI-鼠标也可以视为一组观察者-被观察对象：鼠标**发布**一个行为（点击、滚轮等）时，GUI的各个组件（可以认为是**订阅/观察**了该鼠标的行为）收到**通知**（鼠标点击的位置、左键还是右键等），并自动**更新**（如某个按钮被按下后，更新界面的显示内容）

<span style="color:red; font-weight:bold;">思考</span>：发布-订阅模式的更多使用场景？

本题要求实现一个简易的教学系统
- 学生可以选课或退课
- 教师可以在其负责的课程中发布信息
- 教师发布信息后，选课的学生可以收到信息

需要补充的内容包括
- `Course.add_student`函数：模拟学生选课，应保证已选课学生不会重复选
- `Course.remove_student`函数：模拟学生退课，应先检查的学生是否在选课名单里
- `Course.notify_students`函数：给所有学生发送通知
- `Teacher.post_message`函数：模拟教师给课程发布通知，应检查教师是否讲授该课程


In [11]:
class Course:
    def __init__(self, course_name):
        self.course_name = course_name
        self.students = [] 
    
    def add_student(self, student):
        if student not in self.students: self.students.append(student)
        return 
        raise NotImplementedError
    
    def remove_student(self, student):
        if student in self.students: self.students.remove(student)
        return 
        raise NotImplementedError
    
    def notify_students(self, message):
        for x in self.students: x.receive_message(self.course_name, message)
        return
        raise NotImplementedError

class Student:
    def __init__(self, name, stu_id):
        self.name = name 
        self.stu_id = stu_id
    
    def receive_message(self, course_name, message):
        print(f"{self.name} 收到了 {course_name} 课程的通知: {message}")

class Teacher:
    def __init__(self, name, courses):
        self.name = name
        self.courses = courses
    
    def post_message(self, course, message):
        if course in self.courses: course.notify_students(message)
        return 
        raise NotImplementedError
            


python_course = Course('Python 程序设计与数据科学导论')
ds_course = Course('数据结构与算法')

teacher_hu = Teacher('胡老师', [python_course])
teacher_wang = Teacher('王老师', [ds_course])

student1 = Student('小明', '001')
student2 = Student('小红', '002')
student3 = Student('小刚', '003')

# 模拟学生选课退课
python_course.add_student(student1)
python_course.add_student(student2)
python_course.add_student(student2)
python_course.add_student(student3)
ds_course.add_student(student2)  
ds_course.add_student(student3) 
ds_course.remove_student(student1)
assert len(python_course.students) == 3
assert len(ds_course.students) == 2

# 教师发布信息，应有三行输出
teacher_hu.post_message(python_course, "Python课程第一课：数据类型")
print()

# 教师发布信息，应有两行输出
teacher_wang.post_message(ds_course, "数据结构与算法第一课：线性表")
print()

# 教师发布信息，并非其负责课程，不应输出
teacher_wang.post_message(python_course, "Python课程第二课：函数")
print()

# 学生退课
python_course.remove_student(student3)
assert len(python_course.students) == 2

# 教师发布信息，应有两行输出
teacher_hu.post_message(python_course, "Python课程第二课：函数")
print()

小明 收到了 Python 程序设计与数据科学导论 课程的通知: Python课程第一课：数据类型
小红 收到了 Python 程序设计与数据科学导论 课程的通知: Python课程第一课：数据类型
小刚 收到了 Python 程序设计与数据科学导论 课程的通知: Python课程第一课：数据类型

小红 收到了 数据结构与算法 课程的通知: 数据结构与算法第一课：线性表
小刚 收到了 数据结构与算法 课程的通知: 数据结构与算法第一课：线性表


小明 收到了 Python 程序设计与数据科学导论 课程的通知: Python课程第二课：函数
小红 收到了 Python 程序设计与数据科学导论 课程的通知: Python课程第二课：函数



实现求和计算器`SumCalculator`，使得
- 首次调用，打印问候语`"Give me integers to add them up"`并返回`None`
- 之后每次通过`send`传入
  - 整数：返回过去收到的所有整数的和
  - `None`：停止运行
  - 其他情况：打印`"I only accept integers"`，并返回现有的整数和

## **1.3** `yield`生成器

In [ ]:
def SumCalculator():
    print("Give me integers to add them up")
    total = yield
    while True:
        v = yield total
        if v == None: return # 注意要直接return来触发StopIteration，不要用yield
        if not isinstance(v, int): print("I only accept integers")
        else: total += v

repeater = SumCalculator()
assert next(repeater) is None   # 初次调用，应该打印问候语
assert repeater.send(1) == 1
assert repeater.send(2) == 3
assert repeater.send(3.) == 3   # 传入非整数，应该打印错误信息
assert repeater.send(3) == 6
assert repeater.send('4') == 6  # 传入非整数，应该打印错误信息
try:
    repeater.send(None)
except StopIteration:
    pass
else:
    assert False

Give me integers to add them up
I only accept integers
I only accept integers
